# MQTT Subscribe 測試應用程式

這個 notebook 示範如何使用 paho-mqtt 套件來訂閱和接收 MQTT 訊息。


In [2]:
import paho.mqtt.client as mqtt
import time
import json


In [3]:
# MQTT Broker 設定
BROKER_HOST = "localhost"  # 連接到本地的 Raspberry Pi MQTT broker
# 或者使用 IP 地址（如果從其他設備連接）: "192.168.137.37"
BROKER_PORT = 1883  # 標準 MQTT 端口
TOPIC = "客廳/溫度"  # 訂閱的主題名稱（與 lesson6_1.ipynb 相同）

# 可選：如果需要認證，取消註解並填入
# USERNAME = "your_username"
# PASSWORD = "your_password"


In [4]:
# 建立 MQTT 客戶端（使用最新的回調 API 版本 2）
client = mqtt.Client(callback_api_version=mqtt.CallbackAPIVersion.VERSION2)

# 可選：設定認證（如果有需要的話）
# client.username_pw_set(USERNAME, PASSWORD)

# 連接回調函數（API 版本 2 的簽名）
def on_connect(client, userdata, flags, reason_code, properties):
    if reason_code.is_failure:
        print(f"❌ 連接失敗，錯誤代碼: {reason_code}")
    else:
        print(f"✅ 成功連接到 MQTT Broker: {BROKER_HOST}")
        # 連接成功後自動訂閱主題
        print(f"📡 正在訂閱主題: {TOPIC}")
        client.subscribe(TOPIC, qos=1)

# 訂閱回調函數（API 版本 2 的簽名）
def on_subscribe(client, userdata, mid, reason_codes, properties):
    print(f"✅ 成功訂閱主題，訊息 ID: {mid}")

# 接收訊息回調函數（API 版本 2 的簽名）
def on_message(client, userdata, message):
    topic = message.topic
    payload = message.payload.decode('utf-8')
    qos = message.qos
    
    print(f"\n📨 收到訊息:")
    print(f"   主題: {topic}")
    print(f"   QoS: {qos}")
    print(f"   內容: {payload}")
    
    # 嘗試解析 JSON 格式的訊息
    try:
        data = json.loads(payload)
        print(f"   JSON 解析成功:")
        print(f"   {json.dumps(data, indent=6, ensure_ascii=False)}")
    except json.JSONDecodeError:
        # 如果不是 JSON 格式，就當作普通文字處理
        print(f"   (文字訊息)")

# 設定回調函數
client.on_connect = on_connect
client.on_subscribe = on_subscribe
client.on_message = on_message


In [ ]:
# 連接到 MQTT Broker
print(f"🔗 正在連接到 {BROKER_HOST}:{BROKER_PORT}...")
client.connect(BROKER_HOST, BROKER_PORT, 60)

# 開始網路循環（在背景處理訊息）
client.loop_start()

# 等待連接建立和訂閱完成
time.sleep(2)
print("\n🎧 開始監聽訊息...")
print("   (可以執行 lesson6_1.ipynb 來發送測試訊息)")
print("   (或使用 MQTTX 發送訊息到主題 '客廳/溫度')\n")


🔗 正在連接到 localhost:1883...
✅ 成功連接到 MQTT Broker: localhost
📡 正在訂閱主題: 客廳/溫度
✅ 成功訂閱主題，訊息 ID: 2

🎧 開始監聽訊息...
   (可以執行 lesson6_1.ipynb 來發送測試訊息)
   (或使用 MQTTX 發送訊息到主題 '客廳/溫度')




📨 收到訊息:
   主題: 客廳/溫度
   QoS: 0
   內容: Hello MQTT from Raspberry Pi!
   (文字訊息)

📨 收到訊息:
   主題: 客廳/溫度
   QoS: 1
   內容: {"device": "Raspberry Pi", "temperature": 25.5, "humidity": 60, "timestamp": 1764475100.0102003}
   JSON 解析成功:
   {
      "device": "Raspberry Pi",
      "temperature": 25.5,
      "humidity": 60,
      "timestamp": 1764475100.0102003
}

📨 收到訊息:
   主題: 客廳/溫度
   QoS: 0
   內容: 測試訊息 #1 - 2025-11-30 11:58:27
   (文字訊息)

📨 收到訊息:
   主題: 客廳/溫度
   QoS: 0
   內容: 測試訊息 #2 - 2025-11-30 11:58:28
   (文字訊息)

📨 收到訊息:
   主題: 客廳/溫度
   QoS: 0
   內容: 測試訊息 #3 - 2025-11-30 11:58:29
   (文字訊息)

📨 收到訊息:
   主題: 客廳/溫度
   QoS: 0
   內容: 測試訊息 #4 - 2025-11-30 11:58:30
   (文字訊息)

📨 收到訊息:
   主題: 客廳/溫度
   QoS: 0
   內容: 測試訊息 #5 - 2025-11-30 11:58:31
   (文字訊息)


In [9]:
# 持續監聽訊息（30 秒）
# 在這段時間內，任何發送到訂閱主題的訊息都會被接收並顯示
listen_duration = 30  # 監聽時間（秒）
print(f"⏱️  將監聽 {listen_duration} 秒...")
print("   (可以修改 listen_duration 變數來調整監聽時間)\n")

start_time = time.time()
while time.time() - start_time < listen_duration:
    time.sleep(1)
    elapsed = int(time.time() - start_time)
    if elapsed % 5 == 0:  # 每 5 秒顯示一次倒數
        remaining = listen_duration - elapsed
        print(f"⏳ 還剩 {remaining} 秒...")

print(f"\n✅ 監聽時間結束（已監聽 {listen_duration} 秒）")


⏱️  將監聽 30 秒...
   (可以修改 listen_duration 變數來調整監聽時間)

⏳ 還剩 25 秒...
⏳ 還剩 20 秒...

📨 收到訊息:
   主題: 客廳/溫度
   QoS: 0
   內容: Hello MQTTX from Raspberry Pi!
   (文字訊息)
⏳ 還剩 15 秒...
⏳ 還剩 10 秒...
⏳ 還剩 5 秒...
⏳ 還剩 0 秒...

✅ 監聽時間結束（已監聽 30 秒）


In [10]:
# 停止網路循環並斷開連接
client.loop_stop()
client.disconnect()
print("🔌 已斷開 MQTT 連接")


🔌 已斷開 MQTT 連接


## 使用說明

1. **執行順序**：依序執行上面的 cells
2. **測試方式**：
   - 執行 `lesson6_1.ipynb` 來發送測試訊息
   - 或使用 MQTTX 發送訊息到主題 `客廳/溫度`
3. **調整監聽時間**：修改 Cell 5 中的 `listen_duration` 變數
4. **訂閱多個主題**：可以在 `on_connect` 函數中添加多個 `client.subscribe()` 調用
